# Anthropic Claude

Claude is a family of large language models developed by Anthropic and designed to revolutionize the way you interact with AI. Claude excels at a wide variety of tasks involving language, reasoning, analysis, coding, and more. The models are highly capable, easy to use, and can be customized to suit your needs.

In this notebook, we demonstrate how to use Anthropic Claude model for AgentChat in AG2.

## Features

- Function/tool calling
- Structured Outputs ([Notebook example](https://docs.ag2.ai/docs/use-cases/notebooks/notebooks/agentchat_structured_outputs))
- Token usage and cost correctly as per Anthropic's API costs (as of December 2024)

## Requirements
To use Anthropic Claude with AG2, first you need to install the `ag2[anthropic]` package.

To try out the function call feature of Claude model, you need to install `anthropic>=0.23.1`.


In [ ]:
!pip install ag2["anthropic"]

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen[anthropic]
> ```
> or  
> ```bash
> pip install -U pyautogen[anthropic]
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  


## Set the config for the Anthropic API

You can add any parameters that are needed for the custom model loading in the same configuration list.

It is important to add the `api_type` field and set it to a string that corresponds to the client type used: `anthropic`.

Example:
```
[
    {
        "model": "claude-3-5-sonnet-20240620",
        "api_key": "your Anthropic API Key goes here",
        "api_type": "anthropic",
    },
    {
        "model": "claude-3-sonnet-20240229",
        "api_key": "your Anthropic API Key goes here",
        "api_type": "anthropic",
        "temperature": 0.5,
        "top_p": 0.2, # Note: It is recommended to set temperature or top_p but not both.
        "max_tokens": 10000,
    },
    {
        "model":"claude-3-opus-20240229",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.0",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.1",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-3.0-haiku",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
]
```

### Alternative

As an alternative to the api_key key and value in the config, you can set the environment variable `ANTHROPIC_API_KEY` to your Anthropic API key.

Linux/Mac:
```
export ANTHROPIC_API_KEY="your Anthropic API key here"
```
Windows:
```
set ANTHROPIC_API_KEY=your_anthropic_api_key_here
```

In [1]:
import os

from typing_extensions import Annotated

import autogen

config_list_claude = [
    {
        # Choose your model name.
        "model": "claude-3-5-sonnet-20240620",
        # You need to provide your API key here.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
    }
]

### Alternative Anthropic VertexAI Client (GCP)

To use the Anthropic VertexAI client in AutoGen, you need to configure it for use with Google Cloud Platform (GCP). Ensure you have the necessary project credentials and install the required package.

Configuration

The following configuration example demonstrates how to set up Anthropic VertexAI:

In [ ]:
import os

config_list_vertexai = [
    {
        "model": "claude-3-5-sonnet-20240620-v1:0",
        "gcp_project_id": "your_project_id",
        "gcp_region": "us-west-2",  # Replace with your GCP region
        "gcp_auth_token": None,  # Optional: If not passed, Google Default Authentication will be used
        "api_type": "anthropic",
    }
]

assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_vertexai,
    },
)

### Alternative Anthropic VertexAI Client (Google Default Authentication)

If the `gcp_auth_token` is not provided in the configuration, the client will use Google’s default authentication mechanism. This requires the appropriate credentials to be configured in your environment, such as:

- Service account key: You can set the `GOOGLE_APPLICATION_CREDENTIALS` environment variable to point to your service account key file.
- Cloud Shell or GCP Compute Engine: When running in a GCP-managed environment, default authentication is automatically applied.


Example of setting up the environment variable:

export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-key.json"

This allows seamless integration without explicitly specifying the authentication token in your code.

## Two-agent Coding Example

### Construct Agents

Construct a simple conversation between a User proxy and an ConversableAgent based on Claude-3 model.


In [2]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_claude,
    },
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=1,
)

### Initiate Chat

In [ ]:
user_proxy.initiate_chat(
    assistant, message="Write a python program to print the first 10 numbers of the Fibonacci sequence."
)

## Tool Call Example with the Latest Anthropic API 
Anthropic just announced that tool use is now supported in the Anthropic API. To use this feature, please install `anthropic>=0.23.1`.

### Register the function

In [4]:
@user_proxy.register_for_execution()  # Decorator factory for registering a function to be executed by an agent
@assistant.register_for_llm(
    name="get_weather", description="Get the current weather in a given location."
)  # Decorator factory for registering a function to be used by an agent
def preprocess(location: Annotated[str, "The city and state, e.g. Toronto, ON."]) -> str:
    return "Absolutely cloudy and rainy"

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="What's the weather in Toronto?",
)

## Group Chat Example with both Claude and GPT Agents 

### A group chat with GPT-4 as the judge

In [ ]:
from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent

config_list_gpt4 = [
    {
        # Choose your model name.
        "model": "gpt-4",
        # You need to provide your API key here.
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_type": "openai",
    }
]


config_list_gpt35 = [
    {
        # Choose your model name.
        "model": "gpt-3.5-turbo",
        # You need to provide your API key here.
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_type": "openai",
    }
]

alice = AssistantAgent(
    "Openai_agent",
    system_message="You are from OpenAI. You make arguments to support your company's position.",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

bob = autogen.AssistantAgent(
    "Anthropic_agent",
    system_message="You are from Anthropic. You make arguments to support your company's position.",
    llm_config={
        "config_list": config_list_claude,
    },
)

charlie = AssistantAgent(
    "Research_Assistant",
    system_message="You are a helpful assistant to research the latest news and headlines.",
    llm_config={
        "config_list": config_list_gpt35,
    },
)

dan = AssistantAgent(
    "Judge",
    system_message="You are a judge. You will evaluate the arguments and make a decision on which one is more convincing.",
    llm_config={
        "config_list": config_list_gpt4,
    },
)

code_interpreter = UserProxyAgent(
    "code-interpreter",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    default_auto_reply="",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)


@code_interpreter.register_for_execution()  # Decorator factory for registering a function to be executed by an agent
@charlie.register_for_llm(
    name="get_headlines", description="Get the headline of a particular day."
)  # Decorator factory for registering a function to be used by an agent
def get_headlines(headline_date: Annotated[str, "Date in MMDDYY format, e.g., 06192024"]) -> str:
    mock_news = {
        "06202024": "OpenAI competitor Anthropic announces its most powerful AI yet.",
        "06192024": "OpenAI founder Sutskever sets up new AI company devoted to safe superintelligence.",
    }
    return mock_news.get(headline_date, "No news available for today.")


groupchat = GroupChat(
    agents=[alice, bob, charlie, dan, code_interpreter],
    messages=[],
    allow_repeat_speaker=False,
    max_round=10,
)

manager = GroupChatManager(
    groupchat=groupchat,
    llm_config={
        "config_list": config_list_gpt4,
    },
)

task = "Analyze the potential of OpenAI and Anthropic to revolutionize the field of AI based on today's headlines. Today is 06202024."

user_proxy = UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    default_auto_reply="",
    # is_termination_msg=lambda x: True,
)

user_proxy.initiate_chat(manager, message=task)

### Same group chat with Claude 3.5 Sonnet as the judge

In [ ]:
dan = AssistantAgent(
    "Judge",
    system_message="You are a judge. You will evaluate the arguments and make a decision on which one is more convincing.",
    llm_config={
        "config_list": config_list_claude,
    },
)

groupchat = GroupChat(
    agents=[alice, bob, charlie, dan, code_interpreter],
    messages=[],
    allow_repeat_speaker=False,
    max_round=10,
)

manager = GroupChatManager(
    groupchat=groupchat,
    # is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config={
        "config_list": config_list_gpt4,
    },
)

user_proxy.initiate_chat(manager, message=task)